# Soft-sensor para predecir las concentraciones de producto en el proceso de cracking de etileno


## Carga de datos diarios de variables y concentraciones
#### Carga los datos del archivo "short_data.csv" en un Dataframe de pandas

In [5]:
import os
import utils as u

import utils as u
import pandas as pd


file_name = 'data/short_data.csv'
## lectura de csv

##renombrar columnas


## Selección de la variables independientes y variables objetivo
### Las variables objetivo son : 'H2_B', 'CH4_B', 'C2H4_B', 'C3H6_B'

#### Visualiza con matplotlib (seaborn, plotly) las variables objetivo


In [6]:
import matplotlib.pylab as plt


### Data cleaning
#### Prueba a limpiar los datos y visualiza el resultado

### Visualizacióón de la limpieza de datos

#### Selecciona las variables independientes y el target (es decir X e Y)

## Particionado del conjunto de datos, se usa la regla habitual de 1/3
### La librería Pandas proporciona métdos de validación con este otro tipo Bootstrap
#### Particiona el conjunto de datos con la funcióón "train_test_split"

## Instanciamos el modelo a usar, en este caso Gradient Boosting.
### Pandas proporciona este algoritmo de fácil uso. La parametrización se ha realizado con un método de búsqueda evolutiva

#### Hacer prueba con Gradient boosting, también se puede instanciar otro modelo de forma sencilla, por ejemplo random forest

## Entrenamiento del modelo.
### Es interesante conocer la importancia de cada variable en la regresión
#### Entrenar el modelo e imprimir "feature_importance"

## Evaluación del modelo.
### Extraer el Training error, RMSE y R2 score

In [7]:
from math import sqrt
from sklearn.metrics.regression import mean_squared_error, r2_score


#### Visualización de la predicción en el entrenamiento

In [8]:
import seaborn as sns;


### Extrae el error de test RMSE y R2 score

#### Visualización de la predicción de test